In [2]:
#!g2.1
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPooling3D, UpSampling2D, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import AdamW

2023-10-15 05:44:50.201702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 05:44:53.818534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
#!g2.1
X_train = np.load('../y_smp_train.npy')
y_train = np.load('../pars_smp_train.npy')
X_test = np.load('../y_smp_test.npy')

In [5]:
X_train.shape, y_train.shape

((1000000, 200, 3), (1000000, 15, 1))

In [17]:
from tensorflow.keras import backend as K

In [13]:
def quantile_loss_1(y_true, y_pred):
    error = y_true - y_pred
    loss = K.mean(K.maximum(0.1 * error, (0.1 - 1) * error))
    return loss

In [14]:
#!g2.1
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(200, 3, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=quantile_loss_1, optimizer=opt, metrics=[RootMeanSquaredError()])

In [15]:
#!g2.1
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 200, 3, 16)        160       
                                                                 
 up_sampling2d_2 (UpSampling  (None, 400, 6, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 400, 6, 32)        4640      
                                                                 
 up_sampling2d_3 (UpSampling  (None, 800, 12, 32)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 132, 7, 32)        73760     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 66, 4, 32)       

In [18]:
#!g2.1
model.fit(X_train, y_train, batch_size=128, epochs=25)

Epoch 1/25


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [10]:
#!g2.1
pred = model.predict(X_test)

3125/3125 [==============================] - 4s 1ms/step


In [11]:
#!g2.1
np.save('test_pred_mean', pred)

In [12]:
#!g2.1
model.save("model_mean.h5")

In [ ]:
#!g2.1
